In [68]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
import functions as fn
import psycopg2
from dotenv import load_dotenv
from pymongo import MongoClient
import os



In [ ]:
data = fn.dt_get_data()

# === Exibir resultado ===
print("\n=== DataFrame Final ===")
display(data.head())
print("\nColunas no DataFrame final:")
print(data.columns)



INFO: Tentando conectar DB1 (DESTINO): Host=pg-intersegundo-intercalbon.h.aivencloud.com, DB=bancosegundoano, User=avnadmin, Port=17807
INFO: Conexão DB1 (DESTINO) estabelecida com sucesso.


C:\Users\allanaabrego-ieg\AppData\Local\Temp\ipykernel_14500\1932542841.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  merged_sql = pd.read_sql(sql_query, conn_1)



=== DataFrame Final ===
   nivel_emissao classificacao_emissao nivel_cargo nome_categoria
0           3.23                 Médio        Alto           Quis
1           4.99                  Alto       Médio       Adipisci
2           1.42                 Baixo       Médio     Architecto
3           2.42                 Baixo        Alto             At
4           2.67                 Médio        Alto           Quos

Colunas no DataFrame final:
Index(['nivel_emissao', 'classificacao_emissao', 'nivel_cargo',
       'nome_categoria'],
      dtype='object')


In [70]:
X, y = fn.separate_features_and_target(data, 'classificacao_emissao')
y_encoded = LabelEncoder().fit_transform(y)

In [71]:
df_num_columns = fn.get_data_numeric(X)
df_cat_columns = fn.get_data_string(X, 'classificacao_emissao')

print("Colunas Numéricas:", df_num_columns)
print("Colunas Categóricas:", df_cat_columns)

Colunas Numéricas: ['nivel_emissao']
Colunas Categóricas: ['nivel_cargo', 'nome_categoria']


In [72]:

preprocessor = fn.preprocess_data(df_num_columns, df_cat_columns)


In [73]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", KNeighborsClassifier())
])

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

In [75]:
display(X_train)
display(y_train)
display(X_test)
display(y_test)

,nivel_emissao,nivel_cargo,nome_categoria
724,4.46,Alto,Excepturi
765,4.81,Baixo,Consequatur
3796,1.27,Alto,Rerum
3858,2.16,Médio,Dolorem
307,4.80,Médio,Quidem
...,...,...,...
3772,1.97,Alto,Praesentium
5191,2.45,Médio,Architecto
5226,3.50,Médio,Iusto
5390,3.10,Baixo,Sunt


array([0, 0, 1, ..., 0, 2, 2], shape=(4970,))

,nivel_emissao,nivel_cargo,nome_categoria
4977,3.24,Alto,Sunt
3305,1.97,Médio,Indústria
4391,3.07,Baixo,Voluptatem
5799,2.68,Alto,Aut
3795,1.92,Alto,Indústria
...,...,...,...
336,4.92,Alto,Expedita
4985,3.27,Baixo,Sapiente
3049,4.37,Alto,Adipisci
5058,2.92,Alto,Dolore


array([2, 1, 2, ..., 0, 2, 2], shape=(2130,))

In [76]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [77]:
y_pred

array([2, 1, 2, ..., 0, 2, 2], shape=(2130,))

In [78]:
print(classification_report(y_test, y_pred, target_names=LabelEncoder().fit(y).classes_))

              precision    recall  f1-score   support

        Alto       0.87      0.89      0.88       806
       Baixo       0.87      0.88      0.87       779
       Médio       0.76      0.72      0.74       545

    accuracy                           0.84      2130
   macro avg       0.83      0.83      0.83      2130
weighted avg       0.84      0.84      0.84      2130



In [79]:
param_grid = {
    'classifier__n_neighbors': [3, 5, 7, 9],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan','minkowski'], #botei todas as métricas pra ele testar de td
    'classifier__p': [1, 2, 3], #tenho q ver se isso aq vai coisar o modelo
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'classifier__leaf_size': [20, 30, 40, 50], #O padrão para 10k é 30, mas com buscamos o melhor modelo, visei testar outros valores, seguindo o padrão de 5 elementos usado na DecisionTree
    'classifier__n_jobs': [-1] #Usar todos os núcleos disponíveis -> São 10k de dados
}

grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("classification report with best parameters:", classification_report(y_test, grid.predict(X_test), target_names=LabelEncoder().fit(y).classes_))
print("Best cross-validation score:", grid.best_score_)

C:\Users\allanaabrego-ieg\AppData\Roaming\Python\Python313\site-packages\sklearn\neighbors\_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\allanaabrego-ieg\AppData\Roaming\Python\Python313\site-packages\sklearn\neighbors\_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\allanaabrego-ieg\AppData\Roaming\Python\Python313\site-packages\sklearn\neighbors\_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\allanaabrego-ieg\AppData\Roaming\Python\Python313\site-packages\sklearn\neighbors\_base.py:598: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\allanaabrego-ieg\Ap

Best parameters: {'classifier__algorithm': 'auto', 'classifier__leaf_size': 20, 'classifier__metric': 'euclidean', 'classifier__n_jobs': -1, 'classifier__n_neighbors': 9, 'classifier__p': 1, 'classifier__weights': 'distance'}
classification report with best parameters:               precision    recall  f1-score   support

        Alto       0.95      0.95      0.95       806
       Baixo       0.93      0.93      0.93       779
       Médio       0.84      0.84      0.84       545

    accuracy                           0.91      2130
   macro avg       0.91      0.91      0.91      2130
weighted avg       0.91      0.91      0.91      2130

Best cross-validation score: 0.8957746478873239
